In [1]:
import sys
sys.path.append('../')
from utils import *
from model import *

In [2]:
data_dir = '/datadrive/data/'
batch_size = 512
batch_num  = 128
epoch_num  = 100
samp_num   = 7

device = torch.device("cuda:2")
graph = dill.load(open(data_dir + 'graph.pk', 'rb'))

In [3]:
train_range = {t: True for t in graph.times if t != None and t <= 2015}
valid_range = {t: True for t in graph.times if t != None and (t > 2015) & (t < 2018)}
test_range  = {t: True for t in graph.times if t != None and t >= 2018}

In [4]:
def pf_sample(seed, papers, pairs, t_range, batch_size, test = False):
    np.random.seed(seed)
    _time = np.random.choice(list(papers.keys()))
    pids = np.array(papers[_time])[np.random.choice(len(papers[_time]), batch_size, replace = False)]
    fids = []
    edge = defaultdict(lambda: {})
    for x_id, p_id in enumerate(pids):
        f_ids = pairs[p_id]
        for f_id in f_ids:
            if f_id not in fids:
                fids += [f_id]
            edge[x_id][fids.index(f_id)] = True
    pids = np.stack([pids, np.repeat([_time], batch_size)]).T
    fids = np.stack([fids, np.repeat([_time], len(fids))]).T
 
    feature, times, edge_list = sample_subgraph(graph, t_range, \
                inp = {'paper': pids, 'field': fids}, sampled_depth = 4, sampled_number = 128)

    el = []
    for i in edge_list['paper']['field']['rev_PF_in']:
        if i[0] in edge and i[1] in edge[i[0]]:
            continue
        el += [i]
    edge_list['paper']['field']['rev_PF_in'] = el

    el = []
    for i in edge_list['field']['paper']['PF_in']:
        if i[1] in edge and i[0] in edge[i[1]]:
            continue
        el += [i]
    edge_list['field']['paper']['PF_in'] = el
    
    
    node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = \
            to_torch(feature, times, edge_list)
    '''
        Trace the paper_id and field_id by its own index plus the type start index
    '''
    paper_ids = np.arange(len(pids)) + node_dict['paper'][0]
    field_ids = np.arange(len(fids)) + node_dict['field'][0]
    if test:
        ylabel = torch.zeros(batch_size, len(ids))
        for x_id, p_id in enumerate(pids[:,0]):
            for f_id in pairs[p_id]:
                ylabel[x_id][list(ids).index(f_id)] = 1
    else:
        ylabel = torch.zeros(batch_size, len(field_ids))
        for x_id in edge:
            for y_id in edge[x_id]:
                ylabel[x_id][y_id] = 1
        ylabel /= ylabel.sum(axis=1).view(-1, 1)
    return node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel
    
def prepare_data(pool, process_ids):
    jobs = []
    for process_id in process_ids[:-1]:
        p = pool.apply_async(pf_sample, args=(np.random.randint(2**32 - 1), train_papers, \
                                               train_pairs, train_range, batch_size))
        jobs.append(p)
    p = pool.apply_async(pf_sample, args=(np.random.randint(2**32 - 1), valid_papers, \
                                           valid_pairs, valid_range, batch_size))
    jobs.append(p)
    return jobs

In [ ]:
from torch_geometric.nn import GATConv
class GNN(nn.Module):
    def __init__(self, n_hid, n_layers, dropout = 0.5):
        super(GNN, self).__init__()
        self.gcs = nn.ModuleList()
        self.adapt = nn.Linear(n_hid, n_hid // 2)
        self.drop  = nn.Dropout(dropout)
        for l in range(n_layers):
            self.gcs.append(GATConv(n_hid // 2, n_hid // 2 // 4, heads=4))
    def set_device(self, device):
        self.device = device
        for gc in self.gcs:
            gc.device = device
    def forward(self, node_feature, edge_index):
        meta_xs = self.drop(F.elu(self.adapt(node_feature)))
        for gc in self.gcs:
            meta_xs = self.drop(gc(meta_xs, edge_index))
        return meta_xs

In [ ]:
'''
Paper-Field
'''
field_dict = dill.load(open(data_dir + 'field_dict.pk', 'rb'))
ids = np.array([graph.node_forward['field'][k] for k in field_dict if field_dict[k][0] == 'L1'])

paper_ser = {}

train_pairs = {}
valid_pairs = {}
test_pairs  = {}

train_papers = {_time: {} for _time in train_range}
valid_papers = {_time: {} for _time in valid_range}
test_papers  = {_time: {} for _time in test_range}

for f_id in ids:
    for p_id in graph.edge_list['field']['paper']['PF_in'][f_id]:
        _time = graph.edge_list['field']['paper']['PF_in'][f_id][p_id]
        if _time in train_range:
            if p_id not in train_pairs:
                train_pairs[p_id] = []
            train_pairs[p_id] += [f_id]
            train_papers[_time][p_id] = True
        elif _time in valid_range:
            if p_id not in valid_pairs:
                valid_pairs[p_id] = []
            valid_pairs[p_id] += [f_id]
            valid_papers[_time][p_id] = True
        else:
            if p_id not in test_pairs:
                test_pairs[p_id] = []
            test_pairs[p_id] += [f_id]
            test_papers[_time][p_id] = True
for _time in list(train_papers.keys()):
    if len(train_papers[_time]) < batch_size:
        train_papers.pop(_time)
    else:
        train_papers[_time] = np.array(list(train_papers[_time].keys()))
for _time in list(valid_papers.keys()):
    if len(valid_papers[_time]) < batch_size:
        valid_papers.pop(_time)
    else:
        valid_papers[_time] = np.array(list(valid_papers[_time].keys()))
for _time in list(test_papers.keys()):
    if len(test_papers[_time]) < batch_size:
        test_papers.pop(_time)
    else:
        test_papers[_time] = np.array(list(test_papers[_time].keys()))

In [ ]:
gnn = GNN(400, n_layers = 2).to(device)
matcher = Matcher(200, n_heads = 8).to(device)
model = nn.Sequential(gnn, matcher)
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 2000, eta_min=0)

In [ ]:
stats = []
pool = mp.Pool(16)
process_ids = np.arange(batch_num // 8)
st = time.time()
jobs = prepare_data(pool, process_ids)
train_step = 3000
best_val   = 0
res = []
criterion = nn.KLDivLoss(reduction='batchmean')
for epoch in np.arange(epoch_num)+1:
    '''
        Prepare Training and Validation Data
    '''
    train_data = [job.get() for job in jobs[:-1]]
    valid_data = jobs[-1].get()
    pool.terminate()
    pool.join()
    pool = mp.Pool(16)
    jobs = prepare_data(pool, process_ids)
    et = time.time()
    print('Data Preparation: %.1fs' % (et - st))
    
    
    model.train()
    train_losses = []
    for batch in np.arange(8):
        for node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel in train_data:
            node_rep = gnn.forward(node_feature.to(device), edge_index.to(device))
            res  = matcher.forward(node_rep[field_ids], node_rep[paper_ids])
            loss = criterion(res, ylabel.to(device))
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            train_losses += [loss.cpu().detach().tolist()]
            train_step += 1
            scheduler.step(train_step)
    '''
        Valid
    '''
    model.eval()
    with torch.no_grad():
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = valid_data
        node_rep = gnn.forward(node_feature.to(device), edge_index.to(device))
        res  = matcher.forward(node_rep[field_ids], node_rep[paper_ids])
        loss = criterion(res, ylabel.to(device))
        valid_res = []

        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            valid_res += [ai[bi].tolist()]
        valid_ndcg = np.average([ndcg_at_k(resi, len(resi)) for resi in valid_res])
        if valid_ndcg > best_val:
            best_val = valid_ndcg
            torch.save(model, './save/gat.pt')
        st = time.time()
        print(("Epoch: %d (%.1fs)  LR: %.5f Train Loss: %.2f  Valid Loss: %.2f  Valid NDCG: %.4f") % \
              (epoch, (st-et), optimizer.param_groups[0]['lr'], np.average(train_losses), loss.cpu().detach().tolist(),\
              valid_ndcg))

        if epoch % 5 == 0:
            '''
                Test
            '''
            _time = np.random.choice(list(test_papers.keys()))
            field_rep = []
            for batch_ser in tqdm(np.arange(len(ids) // batch_size + 1)):
                batch_fids = ids[batch_ser * batch_size: (batch_ser + 1) * batch_size]
                batch_fids = np.stack([batch_fids, np.repeat([_time], len(batch_fids))]).T
                feature, times, edge_list = sample_subgraph(graph, test_range, \
                                inp = {'field': batch_fids}, sampled_depth = 4, sampled_number = 128)
                node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = \
                            to_torch(feature, times, edge_list)
                field_ids = np.arange(len(batch_fids)) + node_dict['field'][0]
                node_rep  = gnn.forward(node_feature.to(device), edge_index.to(device))
                field_rep += [node_rep[field_ids]]
            field_rep = torch.cat(field_rep)
            node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                           test_pairs, test_range, batch_size, test=True)
            paper_rep = gnn.forward(node_feature.to(device), edge_index.to(device))[paper_ids]
            res = matcher.forward(field_rep, paper_rep)
            test_res = []
            for ai, bi in zip(ylabel, res.argsort(descending = True)):
                test_res += [ai[bi].tolist()]
            test_ndcg = np.average([ndcg_at_k(resi, len(resi)) for resi in test_res])
            print(test_ndcg)

Data Preparation: 76.6s


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 1 (36.6s)  LR: 0.00059 Train Loss: 3.43  Valid Loss: 2.99  Valid NDCG: 0.4236
Data Preparation: 40.6s


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 2 (37.4s)  LR: 0.00068 Train Loss: 3.08  Valid Loss: 2.98  Valid NDCG: 0.4561
Data Preparation: 35.0s


...

/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 6 (37.0s)  LR: 0.00095 Train Loss: 2.69  Valid Loss: 2.11  Valid NDCG: 0.6195
Data Preparation: 41.3s
Epoch: 7 (37.3s)  LR: 0.00098 Train Loss: 2.61  Valid Loss: 2.29  Valid NDCG: 0.5911
Data Preparation: 36.6s
Epoch: 8 (36.6s)  LR: 0.00100 Train Loss: 2.56  Valid Loss: 2.34  Valid NDCG: 0.5789
Data Preparation: 42.1s
Epoch: 9 (36.9s)  LR: 0.00100 Train Loss: 2.54  Valid Loss: 2.34  Valid NDCG: 0.5671
Data Preparation: 38.5s
Epoch: 10 (38.2s)  LR: 0.00098 Train Loss: 2.50  Valid Loss: 2.35  Valid NDCG: 0.6026



0.5206210048167037
Data Preparation: 49.7s


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 11 (36.7s)  LR: 0.00094 Train Loss: 2.36  Valid Loss: 2.17  Valid NDCG: 0.6267
Data Preparation: 40.1s
Epoch: 12 (37.0s)  LR: 0.00089 Train Loss: 2.36  Valid Loss: 2.23  Valid NDCG: 0.6009
Data Preparation: 37.9s
Epoch: 13 (37.2s)  LR: 0.00082 Train Loss: 2.35  Valid Loss: 2.14  Valid NDCG: 0.6070
